<a href="https://colab.research.google.com/github/ChrisHallak/LLM-Fine-Tunning/blob/main/Qwen_model_finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/gdrive")

Mounted at /gdrive


# Install required packages

In [ ]:
!pip install -qU datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install PyPDF2
!pip install transformers -U
!pip install bitsandbytes -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Log in into Hungging Face account and WandB biases

In [ ]:
from google.colab import userdata
import wandb

huggingface_key = userdata.get('huggingface')
wandb_key = userdata.get('wandb')
openai_key = userdata.get('openai')

wandb.login()
!huggingface-cli login --token {huggingface_key}


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: christeenhallak33 (christeenhallak33-coretech-mena) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingfaceToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingfaceToken`


# Import Packages

In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime
import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

data_dir = '/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/question_dataset_1000.jsonl'
saved_dir = '/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning'
gbase_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

# Upload local PDF

In [ ]:

# Step 1: Upload file from your local device
from google.colab import files
uploaded = files.upload()  # Opens file picker

# Step 2: Extract the file path (assumes single file uploaded)
import os

pdf_path = next(iter(uploaded))  # Get the uploaded filename


Saving history of Oman.pdf to history of Oman.pdf


# Extract Text From the PDF

In [ ]:
# Step 3: Use your existing function (adjusted to skip URL handling)
import PyPDF2

def extract_text_from_pdf_local(pdf_path: str) -> str:
    """Extract text from a local PDF file."""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            pages = reader.pages
            pdf_text = " ".join(page.extract_text() or "" for page in pages).strip()
            return pdf_text
    except Exception as e:
        return f"Error reading PDF: {e}"

# Step 4: Extract text from the uploaded PDF
pdf_content_local = extract_text_from_pdf_local(pdf_path)

pdf_content = ' '.join(pdf_content_local.split())
print(pdf_content)

التراث والتاريخ العريق في سلطنة عُمان مقدمة سلطنة عُمان، الواقعة في الركن الجنوبي الشرقي من شبه الجزيرة العربية، تُعد واحدة من أقدم الدول التي استوطنتها الحضارات البشرية، ولعبت دورًا مهمًا في تاريخ المنطقة والعالم. بفضل موقعها الجغرافي الفريد، المُطل على ممرات بحرية حيوية، أصبحت مركزًا للتجارة والمالحة منذ آالف السنين. وال يقتصر تميز عُمان على موقعها فقط، بل يمتد إلى تراثها الغني الذي يجمع بين عبق التاريخ وروح األصالة. عُمان عبر العصور عُمان القديمة يعود تاريخ االستيطان البشري في عمان إلى ما قبل أكثر من 10,000 سنة، حيث دلت الحفريات األثرية على وجود حضارات قديمة مثل حضارة "مجان"، التي ذكرتها النصوص السومرية وكانت تشتهر بتجارة النحاس والديوريت. العصور اإلسالمية مع بداية القرن السابع الميالدي، دخل اإلسالم إلى عُمان طوعًا، وكان العمانيون من أوائل من اعتنقوا اإلسالم دون قتال. وقد ساهمت عُمان في نشر اإلسالم في شرق أفريقيا والهند وجنوب شرق آسيا عن طريق تجارها وبحّارتها. الدولة اليعربية والنهضة البحرية في القرن السابع عشر، برزت الدولة اليعربية كقوة بحرية عُمانية استطاعت طرد البرتغاليين من سواح

# Define Model Output Schema

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict, Literal


# --- Property choices ---
DifficultyLevel = Literal["Very Easy", "Easy", "Average", "Difficult"]
# BloomTaxonomy = Literal["Remember", "Apply", "Evaluate"]
# LanguageType = Literal["Arabic", "English"]
# ResponseTime = Literal["Short", "Medium", "Long"]



# --- Properties for a single question ---
class QuestionProperties(BaseModel):
    difficulty: DifficultyLevel = Field(..., alias="Difficulty")
    # bloom_taxonomy: BloomTaxonomy = Field(..., alias="Bloom Taxonomy")
    # language: LanguageType = Field(..., alias="Language")
    # response_time: ResponseTime = Field(..., alias="Response Time")


class Question(BaseModel):
    question_text: str = Field(..., description="Question text.")
    question_answers: List[str] = Field(..., min_items=2, description="List of answer options.")
    correct_answer: str = Field(..., description="The correct answer.")
    properties: QuestionProperties = Field(..., description="Per-question attributes.")


# --- Full question set schema ---
class QuestionSet(BaseModel):
    questions: List[Question] = Field(..., description="List of MCQs")


# Generate Dynamic Prompt based on parameters like
###1.Number of Question
###2.Questions Type
###3.Number of Choices
###4.Difficulty Distribution
###5.Language of questoins

In [ ]:
def generate_prompt(number_of_questions, question_type, number_of_choices, properties, language):
    QUESTION_TYPE_MAP = {
        0: "multiple-choice questions (one correct answer)",
        1: "multiple-choice questions (multiple correct answers)",
        2: "true/false questions",
        3: "essay questions"
    }

    property_distributions = {}
    question_properties_lines = []
    total_questions_assigned = 0
    processed_items = []

    # First pass: calculate question counts from percentages and collect fixed values
    for prop_name, prop_value in properties.items():
        if isinstance(prop_value, list) and all(isinstance(item, dict) for item in prop_value):
            dist_parts = []
            enum_parts = []
            for item in prop_value:
                for k, v in item.items():
                    if isinstance(v, (int, float)):
                        count = int((v * number_of_questions) / 100)
                        processed_items.append((prop_name, k, v, count))
                        total_questions_assigned += count
                    else:
                        # Handle non-percentage items if necessary, although the current logic assumes percentage for lists of dicts
                         pass
            if dist_parts:
                property_distributions[prop_name] = ", ".join(dist_parts)
                question_properties_lines.append(
                    f'       "{prop_name}": {" | ".join([k.strip("") for k in enum_parts])}'
                )
        else:
            # Fixed value
            property_distributions[prop_name] = str(prop_value)
            question_properties_lines.append(f'       "{prop_name}": "{prop_value}"')


    # Adjust remainder to ensure total = number_of_questions for percentage-based distributions
    remainder = number_of_questions - total_questions_assigned
    if processed_items and remainder > 0:
        # Find the item with the largest initial count to add the remainder to
        processed_items.sort(key=lambda x: x[3], reverse=True)
        prop_name, k, v, count = processed_items[0]
        processed_items[0] = (prop_name, k, v, count + remainder)

    # Rebuild distribution strings and property lines based on adjusted counts
    property_distributions = {}
    question_properties_lines = []

    for prop_name, prop_value in properties.items():
         if isinstance(prop_value, list) and all(isinstance(item, dict) for item in prop_value):
            dist_parts = []
            enum_parts = []
            for item_prop_name, k, v, count in processed_items:
                if item_prop_name == prop_name:
                    if count > 0:
                         dist_parts.append(f'{k} ({count})')
                         enum_parts.append(f'"{k}"')
            if dist_parts:
                property_distributions[prop_name] = ", ".join(dist_parts)
                question_properties_lines.append(
                    f'       "{prop_name}": {" | ".join([k.strip("") for k in enum_parts])}'
                )
         else:
            property_distributions[prop_name] = str(prop_value)
            question_properties_lines.append(f'       "{prop_name}": "{prop_value}"')


    # Section for distribution description
    distribution_lines = "\n".join([
        f"   - {prop}: {desc}" for prop, desc in property_distributions.items() if prop != "extra_properties"
    ])

    # Properties block inside each question
    question_properties_block = ",\n".join(question_properties_lines)
    question_type_str = QUESTION_TYPE_MAP.get(question_type, "questions")

    # Final prompt
    prompt = f"""
You are a question generation expert.
You will be provided by a PDF content associated with a Pydantic scheme.,
generate exactly {number_of_questions} {question_type_str}.
Follow these instructions precisely:

1. Distribute the {number_of_questions} questions according to these constraints:
{distribution_lines}
follow the distribution proportions strictly.

2. Language of the output must be in {language}

⚠️ Do NOT include any explanation, formatting, or text outside the JSON object.
⚠️ Ensure the distribution rules are strictly followed. Extract content accurately and comply with the format.
""".strip()

    return prompt

In [ ]:

user_prompt = generate_prompt(number_of_questions = 5,question_type=0,number_of_choices=3,language='Arabic',properties={
    "Difficulty":[{"Easy":30,"Average":40,"Difficult":30}],
  })

# Evaluate the BaseModel(Qwen-1.2b)

In [ ]:
import json
messages = [
    {"role": "system","content":user_prompt},
    {
        "role": "user",
        "content": "\n".join([
            "## PDF content:",
            pdf_content.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                QuestionSet.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Generated Questions:",
            "```json"
        ])
    }
]

In [ ]:
torch_dtype = None
base_model = AutoModelForCausalLM.from_pretrained(
    gbase_model_id,
    device_map = "auto",
    torch_dtype = torch_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(gbase_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = base_model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
json.loads(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[{'question_text': 'عُمان تُعد واحدة من أقدم الدول التي استوطنتها الحضارات البشرية، كيف يمكن أن نحدد هذا التاريخ؟',
  'question_answers': ['منذ حوالي 10,000 سنة',
   'منذ حوالي 8,000 سنة',
   'منذ حوالي 6,000 سنة',
   'منذ حوالي 4,000 سنة'],
  'correct_answer': 'منذ حوالي 10,000 سنة',
  'properties': {}},
 {'question_text': 'عُمان تُعتبر مركزًا للتجارة والمالحة منذ آلف السنين، كيف يمكن أن نفهم هذا التاريخ؟',
  'question_answers': ['منذ القرن الأول الميلادي',
   'منذ القرن الثاني الميلادي',
   'منذ القرن الثالث الميلادي',
   'منذ القرن الرابع الميلادي'],
  'correct_answer': 'منذ القرن الرابع الميلادي',
  'properties': {}},
 {'question_text': 'عُمان عبر العصور عُمان القديمة، كيف يمكن أن نفهم هذا التاريخ؟',
  'question_answers': ['عبر العصور الإسلامية',
   'عبر العصور الأسلامية',
   'عبر العصور الإسلامية واليهودية',
   'عبر العصور الإسلامية واليهودية والبوذية'],
  'correct_answer': 'عبر العصور الإسلامية',
  'properties': {}},
 {'question_text': 'عُمان تُعتبر مركزًا تجاريًا وسياسيًا تابعًا

# Fine Tunning Qwen Model

## Format FineTunning Dataset

In [ ]:

sft_data_path = data_dir
llm_finetunning_data = []

system_message = "\n".join([
    "You are a question generation expert",
    "You will be provided by a PDF content associated with a Pydantic scheme.,",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# PDF CONTENT:",
            rec["input"],

            "# Task:",
            rec["instruction"],

            "# Output Scheme:",
            json.dumps( QuestionSet.model_json_schema(), ensure_ascii=False ),
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["output"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)
len(llm_finetunning_data)

1175

## Splitting Dataset into Train and Eval Data

In [ ]:
train_sample_sz = 1000

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(saved_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(saved_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(saved_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

## Clonning LLaMA Factory

In [ ]:
!git clone "https://github.com/hiyouga/LLaMA-Factory.git"

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
'''
        "news_finetune_train": {
            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/train.json",
            "columns": {
                "prompt": "instruction",
                "query": "input",
                "response": "output",
                "system": "system",
                "history": "history"
            }
        },
        "news_finetune_val": {
            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/val.json",
            "columns": {
                "prompt": "instruction",
                "query": "input",
                "response": "output",
                "system": "system",
                "history": "history"
            }
        }
'''

# https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

'\n        "news_finetune_train": {\n            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/train.json",\n            "columns": {\n                "prompt": "instruction",\n                "query": "input",\n                "response": "output",\n                "system": "system",\n                "history": "history"\n            }\n        },\n        "news_finetune_val": {\n            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/val.json",\n            "columns": {\n                "prompt": "instruction",\n                "query": "input",\n                "response": "output",\n                "system": "system",\n                "history": "history"\n            }\n        }\n'

## Configuration for Llama factory fine tunning

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
resume_from_checkpoint: /gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/Qwen_models/checkpoint-300
output_dir: /gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/Qwen_models/
logging_steps: 10
save_strategy: steps
save_steps: 50
plot_loss: true
overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 50

report_to: wandb
run_name: questions_generations_Qwen

push_to_hub: true
export_hub_model_id: "Christeen33/questions_generations_Qwen"
hub_private_repo: true
hub_strategy: checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
# # STEP 1: Clone the repo
# !git clone https://github.com/hiyouga/LLaMA-Factory.git

# STEP 2: Change directory
%cd LLaMA-Factory

# STEP 3: Install in editable mode
!pip install -e .

# Force restart the runtime (only works in Colab)
import os
os.kill(os.getpid(), 9)  # This will crash the runtime and force a restart

# After restarting, run only the training command in a new cell:
# !llamafactory-cli train examples/train_lora/news_finetune.yaml

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.
/content/LLaMA-Factory
/bin/bash: line 1: llamafactory-cli: command not found


## Tranning and saving with checkpoints

In [ ]:
%cd /content/LLaMA-Factory

In [ ]:

# STEP 4: Run training using the example config
!llamafactory-cli train examples/train_lora/news_finetune.yaml


# Testing Model After Fine Tunning

In [ ]:
torch_dtype = None
base_model = AutoModelForCausalLM.from_pretrained(
    gbase_model_id,
    device_map = "auto",
    torch_dtype = torch_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(gbase_model_id)

finetuned_model_id = "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/Qwen_models/checkpoint-750"
base_model.load_adapter(finetuned_model_id)

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = base_model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
response

'```json\n"{\\n    \\"questions\\": [\\n        {\\n            \\"question_text\\": \\"ما هي الدولة التي تقع في الركن الجنوبي الشرقي من شبه الجزيرة العربية؟\\",\\n            \\"question_answers\\": [\\n                \\"سلطنة عُمان\\",\\n                \\"دولة الإمارات\\",\\n                \\"البحرين\\",\\n                \\"قطر\\"\\n            ],\\n            \\"correct_answer\\": \\"سلطنة عُمان\\",\\n            \\"properties\\": {\\n                \\"Difficulty\\": \\"Easy\\",\\n                \\"Language\\": \\"Arabic\\"\\n            }\\n        },\\n        {\\n            \\"question_text\\": \\"متى يعود تاريخ الاستيطان البشري في عُمان إلى؟\\",\\n            \\"question_answers\\": [\\n                \\"آلف سنة\\",\\n                \\"4000 سنة atrás\\",\\n                \\"10,000 سنة atrás\\",\\n                \\"6000 سنة atrás\\"\\n            ],\\n            \\"correct_answer\\": \\"10,000 سنة atrás\\",\\n            \\"properties\\": {\\n                \\"Diffi

In [ ]:
cleaned = response.strip()

# Optional: If the model sometimes wraps JSON in markdown (```json ... ```), strip that
if cleaned.startswith("```json"):
    cleaned = cleaned.strip("```json").strip("`").strip()

# Attempt to parse
result = json.loads(cleaned)

In [ ]:
result

'{\n    "questions": [\n        {\n            "question_text": "ما هي الدولة التي تقع في الركن الجنوبي الشرقي من شبه الجزيرة العربية؟",\n            "question_answers": [\n                "سلطنة عُمان",\n                "دولة الإمارات",\n                "البحرين",\n                "قطر"\n            ],\n            "correct_answer": "سلطنة عُمان",\n            "properties": {\n                "Difficulty": "Easy",\n                "Language": "Arabic"\n            }\n        },\n        {\n            "question_text": "متى يعود تاريخ الاستيطان البشري في عُمان إلى؟",\n            "question_answers": [\n                "آلف سنة",\n                "4000 سنة atrás",\n                "10,000 سنة atrás",\n                "6000 سنة atrás"\n            ],\n            "correct_answer": "10,000 سنة atrás",\n            "properties": {\n                "Difficulty": "Easy",\n                "Language": "Arabic"\n            }\n        },\n        {\n            "question_text": "ما هو أحد أشهر 

In [ ]:
json.loads(result)

{'questions': [{'question_text': 'ما هي الدولة التي تقع في الركن الجنوبي الشرقي من شبه الجزيرة العربية؟',
   'question_answers': ['سلطنة عُمان', 'دولة الإمارات', 'البحرين', 'قطر'],
   'correct_answer': 'سلطنة عُمان',
   'properties': {'Difficulty': 'Easy', 'Language': 'Arabic'}},
  {'question_text': 'متى يعود تاريخ الاستيطان البشري في عُمان إلى؟',
   'question_answers': ['آلف سنة',
    '4000 سنة atrás',
    '10,000 سنة atrás',
    '6000 سنة atrás'],
   'correct_answer': '10,000 سنة atrás',
   'properties': {'Difficulty': 'Easy', 'Language': 'Arabic'}},
  {'question_text': 'ما هو أحد أشهر القالع في عُمان؟',
   'question_answers': ['قلعة بهالء',
    'قلعة نزوى',
    'حصن جبرين',
    'قصبة المنصور'],
   'correct_answer': 'قلعة نزوى',
   'properties': {'Difficulty': 'Average', 'Language': 'Arabic'}},
  {'question_text': 'كيف ساهمت عُمان في نشر اإلسالم في العالم؟',
   'question_answers': ['عن طريق التجارة فقط',
    'من خلال الحروب',
    'عن طريق التجارب العلمية',
    'عبر تجاربهم وبحّارتهم'